In [1]:
from z3 import *
from utils import *
from math import ceil
import numpy as np

### Instances

In [2]:
import os

def read_dat_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Read m and n
    m = int(lines[0].strip())
    n = int(lines[1].strip())
    
    # Read l vector
    l = list(map(int, lines[2].strip().split()))
    
    # Read s vector
    s = list(map(int, lines[3].strip().split()))
    
    # Read D matrix
    D = []
    for line in lines[4:]:
        D.append(list(map(int, line.strip().split())))
    
    return {
        'm': m,
        'n': n,
        'l': l,
        's': s,
        'D': D
    }

def read_all_dat_files(directory):
    instances = []
    for filename in os.listdir(directory):
        if filename.endswith('.dat'):
            file_path = os.path.join(directory, filename)
            instance = read_dat_file(file_path)
            instances.append(instance)
    return instances

# Directory containing .dat files
directory = 'instances'

# Read all .dat files and populate instances
instances = read_all_dat_files(directory)

In [3]:
for i in instances:
    max_element = [max(max(row) for row in i['D'])]

max_d = max(max_element)
print(max_d)

339


# MCP

## Metodo 1

In [69]:
from z3 import *

def MCP_a(instance, timeout=5000):
    m = instance['m']
    n = instance['n']
    l = instance['l']
    s = instance['s']
    D = instance['D']  # Distance matrix

    # Variables
    x = [[Bool(f'x_{i+1}_{j+1}') for j in range(n)] for i in range(m)]  # Adjust item names to start from 1
    y = [Int(f'y_{i+1}') for i in range(m)]  # Total size of items assigned to courier i

    # Initialize solver
    solver = Solver()

    # Constraint 1: Each item must be assigned to exactly one courier
    for j in range(n):
        item_assigned = [x[i][j] for i in range(m)]
        solver.add(exactly_one_np(item_assigned, f"item_{j+1}"))  # Adjust item names to start from 1

    # Constraint 2: The total size of items assigned to any courier must not exceed its load capacity
    for i in range(m):
        courier_assigned_size = 0
        for j in range(n):
            courier_assigned_size += If(x[i][j], s[j], 0)
        solver.add(y[i] == courier_assigned_size)
        solver.add(y[i] <= l[i])

    # Constraint 3: Each courier must pick at least one item
    for i in range(m):
        at_least_one_item = Or([x[i][j] for j in range(n)])
        solver.add(at_least_one_item)

    # Constraint 4: Each courier's path must start and end at the depot (location n+1)
    for i in range(m):
        path_starts_at_depot = Or([And(x[i][j], j == n) for j in range(n)])
        path_ends_at_depot = Or([And(x[i][j], j == n) for j in range(n)])
        solver.add(path_starts_at_depot)
        solver.add(path_ends_at_depot)

    # Define the maximum distance variable
    max_distance = Int('max_distance')

    # Distance constraints adjusted for depot start and end
    for i in range(m):
        courier_distance_expr = 0
        for j in range(n):
            courier_distance_expr += D[j][j+1] * If(x[i][j], 1, 0)  # Distance from j to j+1 for items assigned to courier i
        solver.add(max_distance >= courier_distance_expr)

    # Calculate upper_bound without using built-in sum
    upper_bound = 0
    for capacity in l:
        upper_bound += capacity

    # Binary search for minimizing max_distance
    lower_bound = 0

    while lower_bound < upper_bound:
        mid = (lower_bound + upper_bound) // 2
        solver.push()
        solver.add(max_distance <= mid)
        if solver.check() == sat:
            upper_bound = mid
        else:
            lower_bound = mid + 1
        solver.pop()

    # At the end, lower_bound will contain the minimum max_distance
    min_max_distance = lower_bound

    # Add the final constraint to get the actual model
    solver.add(max_distance == min_max_distance)

    # Check for satisfiability
    if solver.check() == sat:
        model = solver.model()
        print("Solution found:")
        for i in range(m):
            assigned_items = [j+1 for j in range(n) if is_true(model[x[i][j]])]  # Adjust item names to start from 1
            print(f"Courier {i+1} assigned items: {assigned_items}")  # Adjust courier names to start from 1
        print(f"Minimum maximum distance: {model[max_distance]}")
    else:
        print("No solution found")

## Metodo 2

In [68]:
def MCP(instance, timeout = 5000):

    m = instance['m']
    n = instance['n']
    l = instance['l']
    s = instance['s']
    D = instance['D']

    # c courier è andato da j a k
    # i righe oggetto
    # j colonne oggetto
    path = [[[Bool(f"p_{c}_{i}_{j}") for j in range(n+1)] for i in range(n+1)] for c in range(m)]

    # considerando
    upperbound = ceil(n/m)*max_d

    def solve_with_max_distance(upperbound, m, n, l, s, D, path):
        solver = Solver()
        solver.set(timeout=timeout) # priority='pareto

        # tutti partono da n+1 e finiscono a n+1
        for c in range(m):
            solver.add(exactly_one_seq([path[c][n+1][j] for j in range(n+1)]))
            solver.add(exactly_one_seq([path[c][i][n+1] for i in range(n+1)]))
        
        # ogni items viene preso da massimo un courier  
        for i in range(n+1):
            for j in range(n+1):
                solver.add(at_most_one_seq([path[c][i][j] for c in range(m)]))
        
        # catena di couriers
        for c in range(m):
            for i in range(n+1):
                for j in range(n+1):
                    solver.add(Implies(path[c][i][j], exactly_one_seq([path[c][j][k] for k in range(n+1)])))

        for c in range(m):
            for j in range(n+1): 
                solver.add(at_most_one_seq([path[c][i][j] for i in range(n+1)]))
    
        for c in range(m):
            for i in range(n+1): 
                solver.add(at_most_one_seq([path[c][i][j] for j in range(n+1)]))

        # Maximum load constraint 
        for c in range(m):
            weigth_set = []
            for i in range(n):
                for k in range(n+1):
                    weigth_set.extend([path[c][i][j]] * s[i])
            solver.add(at_most_k_seq(weigth_set, l[c]))

        # Aggiungere i constraint impliciti------------------------
    
        # Distance constraint ############################################à
        distances = [[] for _ in range(m)] 
        for c in range(m): 
            for i in range(n+1): 
                for j in range(n+1):
                    Implies(path[c][i][j],distances[c].extend([path[c][i][j]] * D[i]))
        
        lengths = [len(array) for array in distances]
        max_length = max(lengths)
        solver.add(max_length <= upperbound)

        if s.check() == sat:
            m = solver.model()
            return (True, [(c, i, j) for i in range(m) for i in range(n+1) for j in range(n) if m.evaluate(path[i][j][k])])
        else:
            return False, []

    #########################################################################à
    lowerbound = n
    best_solution = None
    while lowerbound < upperbound:

        is_sat, solution = solve_with_max_distance(upperbound, m, n, l, s, D, path)
        if is_sat:
            print(f"Found solution with max distance {upperbound}: {solution}")
            best_solution = solution
            upperbound = upperbound - 1
        else:
            print(f"No solution found with max distance {upperbound}")
            upperbound = upperbound - 1
    return best_solution

## Testing

In [72]:
NUM_INST = 2
instance = instances[NUM_INST-1]
print(instance["m"], instance["n"])
MCP_a(instance)

6 9
No solution found
